In [1]:
import polars as pl
from pathlib import Path

In [2]:
df = pl.read_csv("stock_trades/202207-湘财.xls", encoding="gb18030", separator="\t")

In [3]:
def read_df_湘财(f: str | Path) -> pl.DataFrame: 
    df = pl.read_csv(f, encoding="gb18030", separator="\t", infer_schema=False)
    df = df.with_columns(
        pl.selectors.all().str.strip_prefix("=").str.strip_chars('"'),
    ).with_columns(
        pl.col("发生日期").str.to_date("%Y%m%d"),
        # pl.col("证券代码").str.strip_prefix("=").str.strip_chars('"'), 
        pl.col("成交时间").str.to_time(), 
        pl.col("成交数量", "成交价格", "成交金额", "发生金额", "手续费", "印花税", "过户费", "其他费").cast(pl.Float64)
    )
    df = df.filter(
        pl.col("业务名称").is_in(["证券买入", "证券卖出"]), 
    )
    return df

In [4]:
df = pl.DataFrame(
    {
        "foo": ["asldj1", "jjjask", "hasna"],
    }
)
df.with_columns(foo2=pl.col("foo").str.strip_chars("al"))

foo,foo2
str,str
"""asldj1""","""sldj1"""
"""jjjask""","""jjjask"""
"""hasna""","""hasn"""


In [5]:
df = [read_df_湘财(f) for f in Path("stock_trades/").glob("*-湘财.xls")]

In [6]:
d1 = pl.concat(df)

In [7]:
d1 = d1.with_columns(
    券商=pl.lit("湘财"), 
)
d1

发生日期,证券代码,证券名称,买卖标志,业务名称,成交时间,成交数量,成交价格,成交金额,发生金额,手续费,印花税,过户费,其他费,备注,币种,券商
date,str,str,str,str,time,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
2022-07-18,"""002462""","""嘉事堂""","""卖出""","""证券卖出""",09:38:10,-10400.0,13.2062,137344.0,137184.67,21.98,137.35,1.38,0.0,"""证券卖出""","""人民币""","""湘财"""
2022-07-18,"""600408""","""安泰集团""","""买入""","""证券买入""",09:44:52,47000.0,3.19,149930.0,-149955.5,23.99,0.0,1.51,0.0,"""证券买入""","""人民币""","""湘财"""
2022-07-18,"""600648""","""外高桥""","""买入""","""证券买入""",09:44:31,11900.0,12.6066,150019.0,-150044.49,24.0,0.0,1.49,0.0,"""证券买入""","""人民币""","""湘财"""
2022-07-18,"""600269""","""赣粤高速""","""买入""","""证券买入""",09:43:38,40700.0,3.69,150183.0,-150208.53,24.03,0.0,1.5,0.0,"""证券买入""","""人民币""","""湘财"""
2022-07-18,"""600015""","""华夏银行""","""买入""","""证券买入""",09:42:51,30000.0,5.07,152100.0,-152125.86,24.34,0.0,1.52,0.0,"""证券买入""","""人民币""","""湘财"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-06-19,"""603967""","""中创物流""","""卖出""","""证券卖出""",10:18:46,-5000.0,9.13,45650.0,45596.59,7.3,45.65,0.46,0.0,"""证券卖出""","""人民币""","""湘财"""
2023-06-12,"""300641""","""正丹股份""","""卖出""","""证券卖出""",13:22:32,-9600.0,5.05,48480.0,48423.76,7.76,48.48,0.48,0.0,"""证券卖出""","""人民币""","""湘财"""
2023-06-02,"""000655""","""金岭矿业""","""卖出""","""证券卖出""",10:26:35,-7500.0,6.48,48600.0,48543.61,7.78,48.61,0.47,0.0,"""证券卖出""","""人民币""","""湘财"""


In [8]:
def read_df_海通普通(f: str | Path) -> pl.DataFrame: 
    df = pl.read_excel(f, schema_overrides={
        "成交日期": pl.String,
        "成交时间": pl.String,
        "成交数量": pl.Float64, 
        "成交金额": pl.Float64, 
        "印花税": pl.Float64, 
        "其他费": pl.Float64, 
    })
    df = df.filter(
        pl.col("成交时间")!=""
    ).filter(pl.col("操作").is_in(["买", "卖"])
    ).filter(
        (~pl.col("证券代码").str.starts_with("204")) 
        & (~pl.col("证券代码").str.starts_with("1318"))
    ).with_columns(
        pl.col("成交日期").str.to_date("%Y%m%d"),
        pl.col("成交时间").replace({"":None}).str.to_time("%H:%M:%S"),
    )
    return df

In [9]:
df = [ read_df_海通普通(p) for p in Path("stock_trades/").glob("*-海通普通.xlsx")]
df = pl.concat(df)
d2 = df.with_columns(
    券商=pl.lit("海通普通"), 
)
d2.sort("成交日期", "成交时间")

证券代码,证券名称,成交日期,成交时间,成交数量,成交价格,成交金额,发生金额,操作,手续费,印花税,过户费,其他费,备注,券商
str,str,date,time,f64,f64,f64,f64,str,f64,f64,f64,f64,str,str
"""002224""","""三 力 士""",2023-05-23,09:58:07,10800.0,4.59,49572.0,-49576.96,"""买""",4.96,0.0,0.0,0.0,"""三 力 士证券买入""","""海通普通"""
"""300107""","""建新股份""",2023-05-23,10:01:57,10000.0,5.0,50000.0,-50005.0,"""买""",5.0,0.0,0.0,0.0,"""建新股份证券买入""","""海通普通"""
"""603002""","""宏昌电子""",2023-05-24,09:54:48,9800.0,5.06,49588.0,-49593.46,"""买""",4.96,0.0,0.5,0.0,"""宏昌电子证券买入""","""海通普通"""
"""600178""","""东安动力""",2023-05-24,09:59:17,16400.0,6.07,99548.0,-99558.97,"""买""",9.95,0.0,1.02,0.0,"""东安动力证券买入""","""海通普通"""
"""600626""","""申达股份""",2023-05-24,10:06:14,14800.0,3.37,49876.0,-49881.48,"""买""",4.99,0.0,0.49,0.0,"""申达股份证券买入""","""海通普通"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""603386""","""骏亚科技""",2023-07-24,09:38:22,4500.0,11.32,50940.0,50883.51,"""卖""",5.09,50.91,0.49,0.0,"""骏亚科技证券卖出""","""海通普通"""
"""002225""","""濮耐股份""",2023-07-24,09:42:11,13200.0,3.8,50160.0,-50165.02,"""买""",5.02,0.0,0.0,0.0,"""濮耐股份证券买入""","""海通普通"""
"""600735""","""新华锦""",2023-07-24,09:48:27,8200.0,6.07,49774.0,-49779.48,"""买""",4.98,0.0,0.5,0.0,"""新华锦证券买入""","""海通普通"""


In [10]:
df = [ read_df_海通普通(p) for p in Path("stock_trades/").glob("*-海通两融.xlsx")]
df = pl.concat(df)
d3 = df.with_columns(
    券商=pl.lit("海通两融"), 
)
d3.sort("成交日期", "成交时间")

证券代码,证券名称,成交日期,成交时间,成交数量,成交价格,成交金额,发生金额,操作,手续费,印花税,过户费,其他费,备注,券商
str,str,date,time,f64,f64,f64,f64,str,f64,f64,f64,f64,str,str
"""600638""","""新黄浦""",2023-07-26,09:33:56,9300.0,6.526,60696.0,60628.01,"""卖""",6.68,60.7,0.61,0.0,"""新黄浦证券卖出""","""海通两融"""
"""002492""","""恒基达鑫""",2023-07-27,09:30:42,8700.0,6.12,53244.0,53184.91,"""卖""",5.86,53.23,0.0,0.0,"""恒基达鑫证券卖出""","""海通两融"""
"""002136""","""安 纳 达""",2023-07-27,09:32:52,4400.0,11.881,52277.0,52218.97,"""卖""",5.75,52.28,0.0,0.0,"""安 纳 达证券卖出""","""海通两融"""
"""600300""","""维维股份""",2023-08-04,09:36:55,16400.0,3.27,53628.0,53567.91,"""卖""",5.9,53.65,0.54,0.0,"""维维股份证券卖出""","""海通两融"""
"""300241""","""瑞丰光电""",2023-08-04,09:39:38,9700.0,5.13,49761.0,-49766.47,"""买""",5.47,0.0,0.0,0.0,"""瑞丰光电证券买入""","""海通两融"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""002956""","""西麦食品""",2023-10-31,09:31:53,5000.0,14.13,70650.0,70607.91,"""卖""",6.74,35.35,0.0,0.0,"""西麦食品证券卖出""","""海通两融"""
"""603214""","""爱婴室""",2023-10-31,09:39:57,3100.0,15.84,49104.0,-49109.51,"""买""",5.0,0.0,0.51,0.0,"""爱婴室证券买入""","""海通两融"""
"""300132""","""青松股份""",2023-10-31,09:40:55,9600.0,5.21,50016.0,-50021.0,"""买""",5.0,0.0,0.0,0.0,"""青松股份证券买入""","""海通两融"""


In [11]:
d1 = d1.select(
    券商=pl.col("券商"), 
    交易日期=pl.col("发生日期"), 
    交易时间=pl.col("成交时间"), 
    证券代码=pl.col("证券代码"), 
    证券名称=pl.col("证券名称"), 
    买卖标志=pl.col("买卖标志").replace({"证券卖出": "卖出", "证券买入": "买入"}), 
    成交价格=pl.col("成交价格"), 
    成交数量=pl.col("成交数量").abs(), 
    成交金额=pl.col("成交金额"), 
    手续费=pl.col("手续费"), 
    印花税=pl.col("印花税"), 
    过户费=pl.col("过户费"), 
    其他费=pl.col("其他费"), 
    发生金额=pl.col("发生金额"), 
)

In [12]:
d2 = d2.select(
    券商=pl.col("券商"), 
    交易日期=pl.col("成交日期"), 
    交易时间=pl.col("成交时间"), 
    证券代码=pl.col("证券代码"), 
    证券名称=pl.col("证券名称"), 
    买卖标志=pl.col("操作").replace({"卖": "卖出", "买": "买入"}), 
    成交价格=pl.col("成交价格"), 
    成交数量=pl.col("成交数量").abs(), 
    成交金额=pl.col("成交金额"), 
    手续费=pl.col("手续费"), 
    印花税=pl.col("印花税"), 
    过户费=pl.col("过户费"), 
    其他费=pl.col("其他费"), 
    发生金额=pl.col("发生金额"), 
)

In [13]:
d3 = d3.select(
    券商=pl.col("券商"), 
    交易日期=pl.col("成交日期"), 
    交易时间=pl.col("成交时间"), 
    证券代码=pl.col("证券代码"), 
    证券名称=pl.col("证券名称"), 
    买卖标志=pl.col("操作").replace({"卖": "卖出", "买": "买入"}), 
    成交价格=pl.col("成交价格"), 
    成交数量=pl.col("成交数量").abs(), 
    成交金额=pl.col("成交金额"), 
    手续费=pl.col("手续费"), 
    印花税=pl.col("印花税"), 
    过户费=pl.col("过户费"), 
    其他费=pl.col("其他费"), 
    发生金额=pl.col("发生金额"), 
)

In [15]:
df = pl.concat([d1, d2, d3])

In [20]:
df.with_columns(
    成交金额2=pl.col("成交价格") * pl.col("成交数量"),
).with_columns(
    成交金额D=pl.col("成交金额") - pl.col("成交金额2"), 
).with_columns(
    发生金额D=(
        pl.col("发生金额") 
        - (
        pl.when(pl.col("买卖标志") == "买入")
        .then(-pl.col("成交金额"))
        .when(pl.col("买卖标志") == "卖出")
        .then(pl.col("成交金额"))
        - pl.col("手续费")
        - pl.col("印花税")
        - pl.col("过户费")
        - pl.col("其他费")
        )
    ).round(4)
).sort("发生金额D")

券商,交易日期,交易时间,证券代码,证券名称,买卖标志,成交价格,成交数量,成交金额,手续费,印花税,过户费,其他费,发生金额,成交金额2,成交金额D,发生金额D
str,date,time,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""湘财""",2022-07-18,09:44:52,"""600408""","""安泰集团""","""买入""",3.19,47000.0,149930.0,23.99,0.0,1.51,0.0,-149955.5,149930.0,0.0,0.0
"""湘财""",2022-07-18,09:44:31,"""600648""","""外高桥""","""买入""",12.6066,11900.0,150019.0,24.0,0.0,1.49,0.0,-150044.49,150018.54,0.46,0.0
"""湘财""",2022-07-18,09:43:38,"""600269""","""赣粤高速""","""买入""",3.69,40700.0,150183.0,24.03,0.0,1.5,0.0,-150208.53,150183.0,0.0,0.0
"""湘财""",2022-07-18,09:42:51,"""600015""","""华夏银行""","""买入""",5.07,30000.0,152100.0,24.34,0.0,1.52,0.0,-152125.86,152100.0,0.0,0.0
"""湘财""",2022-07-18,09:39:28,"""601992""","""金隅集团""","""卖出""",2.5683,54000.0,138686.0,22.19,138.69,1.38,0.0,138523.74,138688.2,-2.2,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""湘财""",2022-10-24,09:37:27,"""300283""","""温州宏丰""","""买入""",5.4038,12900.0,69709.0,11.15,0.0,0.83,0.0,-69720.15,69709.02,-0.02,0.83
"""湘财""",2022-10-11,09:32:03,"""000900""","""现代投资""","""卖出""",3.8465,34400.0,132321.0,21.17,132.35,1.31,0.0,132167.48,132319.6,1.4,1.31
"""湘财""",2022-07-18,09:38:10,"""002462""","""嘉事堂""","""卖出""",13.2062,10400.0,137344.0,21.98,137.35,1.38,0.0,137184.67,137344.48,-0.48,1.38


In [22]:
df.write_parquet("stock_trades.parquet")

In [23]:
df.write_csv("stock_trades.csv")

In [25]:
df.write_excel("stock_trades.xlsx")